# Sending Messages
This notebook lets you specify recipients to whom you can send user messages. Everyone can log in with their access data and assign this notebook to their HMI. To do this, simply enter your own HMI with the corresponding secret as the client in this script. Running this notebook, you will authorize your HMI to send messages. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
import os
import uuid
import base64
import getpass
import s3i
import time
import requests
import jwt

This Notebook acts as a client in terms of the S³I. In order to use the S³I this notebook needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [ ]:
clientId = input('[S3I]: Please enter the client id:')
clientSecret = getpass.getpass('[S3I]: Please enter the secret:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: Client id and secret are set")

Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the wheel loader's location on your behalf.

In [ ]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: DEMO Wheel loader (service call), please log in!")
username = input('[S3I]: Please enter your username:')
password = getpass.getpass('[S3I]: Please enter the password:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                                   ) + "]: Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                         client_id=clientId, client_secret=clientSecret, username=username, password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN, scope="")

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                               time.localtime()) + "]: Token received " + parsed_username + " logged in.")

# Add Public Key
In order to receive encrypted messages, you have to insert your personal key. The public key part of this key is added to the directory. Participants who want to send encrypted messages to you need the public key to encrypt their message with your public key. 

In [ ]:
key = input('[S3I]: Please enter your key:')
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
dir = s3i.Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
personalKey = s3i.Key(key_str=json.loads(key)["Key"])
personalKey.exportPubKeyToFile(filename="pubKey.asc", path = "key/")
personalKey.exportsecKeyToFile(filename="secKey.asc", path = "key/")
response = dir.putPublicKey(clientId,"key/pubKey.asc")
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                               time.localtime()) + "]: Added public key to the directory entry of", clientId, ".")

# Send User Messages

## Verify the endpoints
You can type in the last names of your respective receiver. The endpoints of his/her HMI is requested from the directory.

In [ ]:
receivers_names = [str(x) for x in input('[S3I]: Please enter the last names of the receivers, seperate with a space:').split()]

In [ ]:
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN, scope="rabbitmq.write:*/*/*")
dir = s3i.Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
receivers_entries = [dir.searchAll(queryParameter="filter=like(attributes/name,\"DT of "+receivers_name+"\")") for receivers_name in receivers_names]
receivers = [receiver["items"][0]["thingId"] for receiver in receivers_entries]
defaultHMIs = [receiver["items"][0]["attributes"]["defaultHMI"] for receiver in receivers_entries]
endpoints = [dir.queryThingIDBased(hmi+"/attributes/allEndpoints")[0] for hmi in defaultHMIs]

print("[S3I]: You specified " , receivers, " as receivers.")
print("[S3I]: The respective endpoints are:" , endpoints)

## Prepare the message
You have to prepare the message you want to send. The user message consists of a subject and a text. Attachments are optional. The endpoint of your HMI is added as the *replyToEndpoint*.

In [ ]:
subject = input('[S3I]: Please enter the subject: \n')
text = input('[S3I]: Please enter the text: \n')
msg_uuid = "s3i:" + str(uuid.uuid4())
replyToEndpoint=None
attachments=None

### Add attachement to message
If you do not want to add an attachment, just leave the input field empty by pressing **RETURN**.

In [ ]:
filename_list = list(map(str, input("[S3I]: Please enter the name of file that you want to send: ").split()))
attachments = list()

for filename in filename_list:
    attachment_dict = dict()
    attachment_dict["filename"] = filename
    with open("attachment_files/"+filename, "rb") as f:
        # encode an attachment file to BASE64 bytes
        base64_data = base64.b64encode(f.read())
        data = base64_data.decode()  # convert byte to str
        attachment_dict["data"] = data
        attachments.append(attachment_dict)

## Assemble User Message
The parts of the user message are assembled.

In [ ]:
message = s3i.UserMessage(sender=clientId, identifier=msg_uuid,receivers=receivers, subject=subject, text=text,
                  replyToEndpoint=replyToEndpoint, attachments=attachments)
print("[S3I]: Your message is: ", message.msg)

## Sign the message
The message gets signed with your private key.

In [ ]:
pgpKey_sec = s3i.Key(path_demo_dir="key/", filename="secKey.asc")
message.sign(pgpKey_sec.key)
print("[S3I]: The signed PGP message is: \n",message.pgpMsg.__str__())

## Encrypt the message
The message gets encrypted with the public keys of your receivers.

In [ ]:
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
dir = s3i.Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
publicKeys_str = dir.getPublicKey(defaultHMIs)
publicKeys = [s3i.Key(key_str=i) for i in publicKeys_str]
message.encrypt(publicKeys)
print("[S3I]: The encrypted PGP message is: \n",message.pgpMsg.__str__())

## Send the message via S³I Broker
The message is send to the S³I broker which transmitts the message to the respective endpoints of the receivers.

In [ ]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: Sending the user mesage to the S³I broker")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/pgp-encrypted', 'Authorization': 'Bearer ' + access_token}
endpoints_str=""
for endpoint in endpoints:
    endpoints_str = endpoints_str+","+endpoint if (len(endpoints_str)!=0) else endpoint
response = requests.post(url="https://broker.s3i.vswf.dev/"+endpoints_str,
                                data=message.pgpMsg.__str__(), headers=headers)
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: ", response.text)